# Introduction
The purpose of this project is to evaluate the house price difference between Houston and Chicago.
Zillow provided Zillow Home Value Index(ZHVI) as a smoothed, seasonally adjusted measure of the typical home value across a given region and housing type.
I focused on single-family residences for my research.

# Section 1: Data Wrangling.

## <span style='color:blue'> Step 1: Extract house price data</span>
Load House price data from following webpage, https://www.zillow.com/research/data/. I used ZHVI Single-Family Homes Time Series in Metro & U.S from 1996-2020. The downloaded file name is called "Metro_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_mon.csv".

In [2]:
#load python packages
import pandas as pd 
import numpy as np 
import os 
import matplotlib.pyplot as plt 
from sklearn.preprocessing import StandardScaler
import requests
import json
import seaborn as sns

In [3]:
os.getcwd()

'C:\\Users\\Grace-2019\\Documents\\Houston_Chicago_house_price'

In [4]:
# read the house price file
file1 = "Metro_zhvi_uc_sfr_tier_0.33_0.67_sm_sa_mon.csv" 
df1 = pd.read_csv(file1, delimiter = ',')

In [5]:
df1.head(10)

,RegionID,SizeRank,RegionName,RegionType,StateName,1996-01-31,1996-02-29,1996-03-31,1996-04-30,1996-05-31,...,2019-09-30,2019-10-31,2019-11-30,2019-12-31,2020-01-31,2020-02-29,2020-03-31,2020-04-30,2020-05-31,2020-06-30
0,102001,0,United States,Country,NaN,106752.0,106784.0,106839.0,106974.0,107126.0,...,242460.0,243283.0,244088.0,244909.0,245771.0,246708.0,247737.0,248783.0,249707.0,250604.0
1,394913,1,"New York, NY",Msa,NY,192273.0,191824.0,191540.0,190990.0,190654.0,...,481362.0,481472.0,481698.0,482269.0,483093.0,484035.0,484956.0,485911.0,486824.0,487870.0
2,753899,2,"Los Angeles-Long Beach-Anaheim, CA",Msa,CA,194120.0,194423.0,194458.0,194605.0,194608.0,...,705972.0,708697.0,710535.0,713159.0,716361.0,721125.0,725613.0,728709.0,728618.0,727713.0
3,394463,3,"Chicago, IL",Msa,IL,164910.0,164604.0,164198.0,163735.0,163116.0,...,246189.0,246205.0,246213.0,246338.0,246574.0,246841.0,247228.0,247432.0,247577.0,247747.0
4,394514,4,"Dallas-Fort Worth, TX",Msa,TX,112171.0,112242.0,112402.0,112720.0,113060.0,...,253838.0,254282.0,254687.0,254912.0,255366.0,256022.0,256977.0,258030.0,259028.0,260182.0
5,394974,5,"Philadelphia, PA",Msa,PA,123205.0,123051.0,122912.0,122671.0,122509.0,...,254062.0,254657.0,255177.0,255870.0,256645.0,257534.0,258135.0,259090.0,260083.0,261581.0
6,394692,6,"Houston, TX",Msa,TX,110240.0,110304.0,110312.0,110452.0,110592.0,...,218869.0,219367.0,219921.0,220480.0,220671.0,221051.0,221694.0,222559.0,223223.0,223709.0
7,395209,7,"Washington, DC",Msa,DC,189204.0,188967.0,188830.0,188540.0,188395.0,...,456607.0,458191.0,459800.0,461076.0,462081.0,462978.0,464225.0,466064.0,467831.0,469815.0
8,394856,8,"Miami-Fort Lauderdale, FL",Msa,FL,125265.0,125439.0,125614.0,125937.0,126218.0,...,345322.0,346312.0,347011.0,347527.0,348243.0,350031.0,351396.0,352685.0,353734.0,354828.0
9,394347,9,"Atlanta, GA",Msa,GA,121975.0,122206.0,122429.0,122922.0,123407.0,...,239395.0,240196.0,240975.0,241936.0,243053.0,244269.0,245396.0,246517.0,247571.0,248908.0


In [6]:
df1.describe().T

,count,mean,std,min,25%,50%,75%,max
RegionID,915.0,414861.361749,83829.704753,102001.0,394547.50,394802.0,395049.5,753929.0
SizeRank,915.0,460.206557,268.414169,0.0,228.50,457.0,689.5,933.0
1996-01-31,430.0,93191.404651,44910.141489,25765.0,62011.25,84616.0,111944.5,322545.0
1996-02-29,432.0,93192.766204,44732.356206,25782.0,62070.00,84691.0,111725.0,321175.0
1996-03-31,433.0,93206.399538,44663.928140,25791.0,62167.00,84701.0,111325.0,319635.0
...,...,...,...,...,...,...,...,...
2020-02-29,915.0,192355.950820,128859.834117,32560.0,117681.00,155440.0,226203.5,1238106.0
2020-03-31,915.0,192959.147541,129379.472267,32492.0,118147.00,155916.0,227227.5,1250734.0
2020-04-30,915.0,193507.190164,129752.783328,32402.0,118073.50,156280.0,228080.5,1256029.0
2020-05-31,915.0,194007.596721,130034.998537,32290.0,117971.00,157017.0,228599.0,1259468.0


In [7]:
df1.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 915 entries, 0 to 914
Columns: 299 entries, RegionID to 2020-06-30
dtypes: float64(294), int64(2), object(3)
memory usage: 2.1+ MB


In [8]:
df1.dtypes

RegionID        int64
SizeRank        int64
RegionName     object
RegionType     object
StateName      object
               ...   
2020-02-29    float64
2020-03-31    float64
2020-04-30    float64
2020-05-31    float64
2020-06-30    float64
Length: 299, dtype: object

In [9]:
# read time series for houston and chicago price data
df_two_city = df1.iloc[[3,6]]
drop_column = ['RegionID','SizeRank','RegionName', 'RegionType','StateName']
df_two_city_ts = df_two_city.drop(drop_column, axis = 1).T
df_two_city_ts.index = pd.to_datetime(df_two_city_ts.index)
df_two_city_ts.rename(columns = {3:'Chicago',6:'Houston'},inplace = True)
df_two_city_ts.index.names = ['Date']
df_two_city_ts

,Chicago,Houston
Date,,
1996-01-31,164910.0,110240.0
1996-02-29,164604.0,110304.0
1996-03-31,164198.0,110312.0
1996-04-30,163735.0,110452.0
1996-05-31,163116.0,110592.0
...,...,...
2020-02-29,246841.0,221051.0
2020-03-31,247228.0,221694.0
2020-04-30,247432.0,222559.0


## <span style='color:blue'> Step 2: Extract Population Data.</span>
Load population data. The data was downloaded from https://www.census.gov/data/datasets/time-series/demo/popest/2010s-total-cities-and-towns.html. Data is in Excel format and has headers.

In [10]:
file2 = "SUB-IP-EST2019-ANNRNK.xlsx"
df2 = pd.read_excel(file2, header = 3)
df2.head(10)

,Unnamed: 0,Unnamed: 1,Census,Estimates Base,2010,2011,2012,2013,2014,2015,2016,2017,2018,2019
0,1,"New York city, New York",8175133,8175031.0,8190209.0,8272948.0,8346693.0,8396091.0,8433806.0,8463049.0,8469153.0,8437478.0,8390081.0,8336817.0
1,2,"Los Angeles city, California",3792621,3793139.0,3795512.0,3820876.0,3851202.0,3881622.0,3909901.0,3938568.0,3963226.0,3975788.0,3977596.0,3979576.0
2,3,"Chicago city, Illinois",2695598,2695652.0,2697477.0,2708114.0,2719141.0,2725731.0,2727066.0,2724344.0,2716723.0,2711069.0,2701423.0,2693976.0
3,4,"Houston city, Texas",2099451,2095517.0,2100280.0,2126032.0,2161593.0,2199391.0,2241826.0,2286908.0,2309544.0,2316750.0,2318573.0,2320268.0
4,5,"Phoenix city, Arizona",1445632,1446691.0,1449038.0,1469796.0,1499274.0,1526491.0,1555445.0,1583690.0,1612199.0,1633560.0,1654675.0,1680992.0
5,6,"Philadelphia city, Pennsylvania",1526006,1526012.0,1528283.0,1540466.0,1551824.0,1558313.0,1565460.0,1571065.0,1576051.0,1580601.0,1583592.0,1584064.0
6,7,"San Antonio city, Texas",1327407,1326161.0,1332299.0,1357120.0,1383075.0,1408339.0,1435456.0,1464043.0,1487843.0,1511154.0,1530016.0,1547253.0
7,8,"San Diego city, California",1307402,1301929.0,1305906.0,1319592.0,1336776.0,1355320.0,1375831.0,1387323.0,1402089.0,1412621.0,1421917.0,1423851.0
8,9,"Dallas city, Texas",1197816,1197658.0,1200350.0,1218282.0,1242115.0,1258835.0,1279098.0,1301329.0,1323916.0,1342479.0,1341802.0,1343573.0
9,10,"San Jose city, California",945942,952528.0,954940.0,970369.0,983530.0,1001279.0,1014273.0,1025980.0,1030242.0,1032335.0,1028020.0,1021795.0


In [11]:
df2.shape

(793, 14)

In [12]:
df2.columns

Index([    'Unnamed: 0',     'Unnamed: 1',         'Census', 'Estimates Base',
                   2010,             2011,             2012,             2013,
                   2014,             2015,             2016,             2017,
                   2018,             2019],
      dtype='object')

In [13]:
#load the merged 20 year population data for later modeling work
pop_raw = pd.read_csv('population_20years_record.csv', index_col = 'Date').dropna(axis = 1)
pop_raw.head()

,Chicago_population,Houston_population
Date,,
2000,2891582,1974324
2001,2874477,1979589
2002,2847266,1990408
2003,2819117,1992464
2004,2786291,1991817


In [14]:
pop_raw.dtypes

Chicago_population    int64
Houston_population    int64
dtype: object

In [15]:
pop_c = pop_raw[['Chicago_population']].astype('object')
pop_h = pop_raw[['Houston_population']].astype('object')

In [16]:
pop_h.head()

,Houston_population
Date,
2000,1974324
2001,1979589
2002,1990408
2003,1992464
2004,1991817


## <span style='color:blue'> Step 3: Extract GDP data.</span>
Pull out GDP data from the BEA (US Bureau of Economic Analysis) API. BEA published economic statistics using industry-standard methods and procedures.

In [18]:
# Method1 Get data from API
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = ''

In [19]:
# Now, call the BEA API and pull out a small sample of the data to get a glimpse
# into the JSON structure that will be returned
url = "https://apps.bea.gov/api/data/?UserID=API_KEY&method=GetParameterValuesFiltered&datasetname=Regional&TargetParameter=LineCode&TableNa"
r = requests.get(url)
json_data = r.json()
json_data

{'BEAAPI': {'Request': {'RequestParam': [{'ParameterName': 'USERID',
     'ParameterValue': 'API_KEY'},
    {'ParameterName': 'METHOD',
     'ParameterValue': 'GETPARAMETERVALUESFILTERED'},
    {'ParameterName': 'TARGETPARAMETER', 'ParameterValue': 'LINECODE'},
    {'ParameterName': 'TABLENA', 'ParameterValue': ''},
    {'ParameterName': 'DATASETNAME', 'ParameterValue': 'REGIONAL'}]},
  'Results': {'Error': {'APIErrorCode': '1',
    'APIErrorDescription': 'Invalid Request - Invalid API UserId.'}}}}

In [20]:
# method2, download data from BEA website.
# read GDP csv files for Chicago
gdp_c_raw = pd.read_csv('GDP_summary_Chicago.csv', header = 4).dropna()
gdp_c_raw.head()

,GeoFips,GeoName,LineCode,Description,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,16980,"Chicago-Naperville-Elgin, IL-IN-WI (Metropolit...",1.0,Real GDP (thousands of chained 2012 dollars),5.149820e+08,5.172946e+08,5.233121e+08,5.366621e+08,5.486324e+08,5.573862e+08,...,5.308302e+08,5.397048e+08,5.474993e+08,561588192.0,5.664995e+08,5.760001e+08,5.890500e+08,5.940608e+08,6.004220e+08,6.115911e+08
1,16980,"Chicago-Naperville-Elgin, IL-IN-WI (Metropolit...",2.0,Chain-type quantity indexes for real GDP,9.170100e+01,9.211300e+01,9.318400e+01,9.556100e+01,9.769300e+01,9.925200e+01,...,9.452300e+01,9.610300e+01,9.749100e+01,100.0,1.008750e+02,1.025660e+02,1.048900e+02,1.057820e+02,1.069150e+02,1.089040e+02
2,16980,"Chicago-Naperville-Elgin, IL-IN-WI (Metropolit...",3.0,Current-dollar GDP (thousands of current dollars),4.016221e+08,4.108182e+08,4.223338e+08,4.453680e+08,4.698541e+08,4.931725e+08,...,5.006113e+08,5.144260e+08,5.329617e+08,561588192.0,5.776643e+08,5.999194e+08,6.277343e+08,6.417297e+08,6.598550e+08,6.894647e+08


In [21]:
#clear the table and drop unneccessary columns
gdp_c = gdp_c_raw.iloc[[0]].drop(columns = ['GeoFips','GeoName','LineCode']).T
gdp_c.head()

,0
Description,Real GDP (thousands of chained 2012 dollars)
2001,5.14982e+08
2002,5.17295e+08
2003,5.23312e+08
2004,5.36662e+08


In [22]:
# take care the header problem
#grab the first row for the header
new_header = gdp_c.iloc[0] 
#take the data less the header row
gdp_c = gdp_c[1:]
#set the header row as the df header
gdp_c.columns = new_header 
gdp_c

Description,Real GDP (thousands of chained 2012 dollars)
2001,5.14982e+08
2002,5.17295e+08
2003,5.23312e+08
2004,5.36662e+08
2005,5.48632e+08
2006,5.57386e+08
2007,5.6412e+08
2008,5.48672e+08
2009,5.3083e+08
2010,5.39705e+08


In [23]:
# Read GDP csv files for Houston
gdp_h_raw = pd.read_csv('GDP_summary_Houston.csv', header = 4).dropna()
gdp_h_raw.head()

,GeoFips,GeoName,LineCode,Description,2001,2002,2003,2004,2005,2006,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
0,26420,"Houston-The Woodlands-Sugar Land, TX (Metropol...",1.0,Real GDP (thousands of chained 2012 dollars),2.973962e+08,2.946361e+08,2.953759e+08,3.208612e+08,3.261306e+08,3.488816e+08,...,3.659897e+08,3.813466e+08,3.907371e+08,404431385.0,4.194667e+08,4.204497e+08,4.427059e+08,4.291340e+08,4.323292e+08,4.453354e+08
1,26420,"Houston-The Woodlands-Sugar Land, TX (Metropol...",2.0,Chain-type quantity indexes for real GDP,7.353400e+01,7.285200e+01,7.303500e+01,7.933600e+01,8.063900e+01,8.626500e+01,...,9.049500e+01,9.429200e+01,9.661400e+01,100.0,1.037180e+02,1.039610e+02,1.094640e+02,1.061080e+02,1.068980e+02,1.101140e+02
2,26420,"Houston-The Woodlands-Sugar Land, TX (Metropol...",3.0,Current-dollar GDP (thousands of current dollars),2.130964e+08,2.112422e+08,2.202297e+08,2.478295e+08,2.674874e+08,2.972121e+08,...,3.246384e+08,3.498830e+08,3.804595e+08,404431385.0,4.237661e+08,4.307262e+08,4.464870e+08,4.304447e+08,4.475212e+08,4.787786e+08


In [24]:
# Drop unneccessary columns and tanspose data
gdp_h = gdp_h_raw.iloc[[0]].drop(columns = ['GeoFips','GeoName','LineCode']).T
gdp_h.head()

,0
Description,Real GDP (thousands of chained 2012 dollars)
2001,2.97396e+08
2002,2.94636e+08
2003,2.95376e+08
2004,3.20861e+08


In [25]:
new_header_h = gdp_h.iloc[0] #grab the first row for the header
gdp_h = gdp_h[1:] #take the data less the header row
gdp_h.columns = new_header_h #set the header row as the df header
gdp_h

Description,Real GDP (thousands of chained 2012 dollars)
2001,2.97396e+08
2002,2.94636e+08
2003,2.95376e+08
2004,3.20861e+08
2005,3.26131e+08
2006,3.48882e+08
2007,3.76389e+08
2008,3.69381e+08
2009,3.6599e+08
2010,3.81347e+08


## <span style='color:blue'> Step 4: Extract Employment data.</span>
Pull GDP and Employment data from the BEA (US Bureau of Economic Analysis) API. BEA published economic statistics using industry-standard methods and procedures.

In [26]:
# download data from BEA website.
# read GDP csv files for Chicago and Houston
job_raw = pd.read_csv('GDP_Personal _income _H _C.csv', header = 4).dropna()
job_raw.head()

,GeoFips,GeoName,LineCode,Description,1995,1996,1997,1998,1999,2000,...,2009,2010,2011,2012,2013,2014,2015,2016,2017,2018
1,16980,"Chicago-Naperville-Elgin, IL-IN-WI (Metropolit...",10.0,Personal income (thousands of dollars),240472614.0,255530533.0,271320981.0,289689942.0,303849857.0,328420542.0,...,408489123.0,418681182.0,439395786.0,464594859.0,474261503.0,503656560.0,528314915.0,535068699.0,552339301.0,580270144.0
2,16980,"Chicago-Naperville-Elgin, IL-IN-WI (Metropolit...",11.0,Nonfarm personal income 1/,240361657.0,255283431.0,271098513.0,289530322.0,303717598.0,328254694.0,...,408296160.0,418460523.0,438884161.0,464259052.0,473552496.0,503354816.0,528274806.0,534847754.0,552211586.0,580204553.0
3,16980,"Chicago-Naperville-Elgin, IL-IN-WI (Metropolit...",12.0,Farm income 2/,110957.0,247102.0,222468.0,159620.0,132259.0,165848.0,...,192963.0,220659.0,511625.0,335807.0,709007.0,301744.0,40109.0,220945.0,127715.0,65591.0
4,16980,"Chicago-Naperville-Elgin, IL-IN-WI (Metropolit...",20.0,Population (persons) 3/,8693383.0,8782253.0,8862719.0,8949190.0,9035654.0,9113234.0,...,9429498.0,9470880.0,9500991.0,9529773.0,9553268.0,9564614.0,9557880.0,9540144.0,9520784.0,9498716.0
5,16980,"Chicago-Naperville-Elgin, IL-IN-WI (Metropolit...",30.0,Per capita personal income (dollars) 4/,27662.0,29096.0,30614.0,32371.0,33628.0,36038.0,...,43320.0,44207.0,46247.0,48752.0,49644.0,52658.0,55275.0,56086.0,58014.0,61089.0


In [27]:
total_emp = job_raw['Description']=='Total employment'

In [28]:
job = job_raw[total_emp].T[4:].rename(columns = {24:'Chicago_Total_Employment',51:'Houston_Total_Employment'})
job.head()

,Chicago_Total_Employment,Houston_Total_Employment
1995,4.98942e+06,2.40245e+06
1996,5.07256e+06,2.47105e+06
1997,5.15979e+06,2.57986e+06
1998,5.29131e+06,2.70405e+06
1999,5.3574e+06,2.73575e+06


In [29]:
job_c = job[['Chicago_Total_Employment']]
job_c.head()

,Chicago_Total_Employment
1995,4.98942e+06
1996,5.07256e+06
1997,5.15979e+06
1998,5.29131e+06
1999,5.3574e+06


In [30]:
job_h = job[['Houston_Total_Employment']]
job_h.head()

,Houston_Total_Employment
1995,2.40245e+06
1996,2.47105e+06
1997,2.57986e+06
1998,2.70405e+06
1999,2.73575e+06


In [31]:
# Create two seperate time series for Chicago and Houston house price
house_c = df_two_city_ts['Chicago']
house_h = df_two_city_ts['Houston']

In [32]:
#resample Chicago and Houston House Price from monthly to yearly to be consisent with population, GDP and employment data.
house_c_a = house_c.resample('A',axis = 0).mean().to_frame()
house_h_a = house_h.resample('A',axis = 0).mean().to_frame()

In [33]:
house_c_a.index = house_c_a.index.strftime('%Y')
house_h_a.index = house_h_a.index.strftime('%Y')

In [34]:
house_h_a

,Houston
Date,
1996,110770.666667
1997,112007.500000
1998,115692.000000
1999,121003.500000
2000,125962.416667
2001,127369.583333
2002,130712.916667
2003,134868.083333
2004,142466.750000


In [35]:
# merge house price and GDP data for Chicago
two_col_c = house_c_a.join(gdp_c, how = 'left', sort = False)
# merge house price, GOP, employment, for Chicago and Houston
three_col_c = two_col_c.join(job_c, how = 'left', sort = False)
# merge house price, GOP, employment, population for Chicago and Houston
data_mod_c = three_col_c.join(pop_c, how = 'left', sort = False)
#data_mod_c=three_col_c
data_mod_c.head(10)
#data_mod_c.to_csv('Chicago_data_for_modeling.csv')


,Chicago,Real GDP (thousands of chained 2012 dollars),Chicago_Total_Employment,Chicago_population
Date,,,,
1996,164324.000000,NaN,5.07256e+06,NaN
1997,163806.750000,NaN,5.15979e+06,NaN
1998,153246.750000,NaN,5.29131e+06,NaN
1999,165807.083333,NaN,5.3574e+06,NaN
2000,177952.333333,NaN,5.46265e+06,NaN
2001,192719.000000,5.14982e+08,5.45027e+06,NaN
2002,207235.166667,5.17295e+08,5.38696e+06,NaN
2003,222490.500000,5.23312e+08,5.38703e+06,NaN
2004,239666.416667,5.36662e+08,5.44623e+06,NaN


In [36]:
data_mod_c['Chicago_population']

Date
1996    NaN
1997    NaN
1998    NaN
1999    NaN
2000    NaN
2001    NaN
2002    NaN
2003    NaN
2004    NaN
2005    NaN
2006    NaN
2007    NaN
2008    NaN
2009    NaN
2010    NaN
2011    NaN
2012    NaN
2013    NaN
2014    NaN
2015    NaN
2016    NaN
2017    NaN
2018    NaN
2019    NaN
2020    NaN
Name: Chicago_population, dtype: object

In [37]:
# merge house price and GDP data for Houston
two_col_h = house_h_a.join(gdp_h, how = 'left', sort = False)
# merge house price, GOP, employment, for Houston
three_col_h = two_col_h.join(job_h, how = 'left', sort = False)
# merge house price, GOP, employment, population data for Houston
data_mod_h = three_col_h.join(pop_h, how = 'left', sort = False)
#optional, write out data to csv file
#data_mod_h=three_col_h
#data_mod_h.to_csv('Houston_data_for_modeling.csv')
data_mod_h.head(10)

,Houston,Real GDP (thousands of chained 2012 dollars),Houston_Total_Employment,Houston_population
Date,,,,
1996,110770.666667,NaN,2.47105e+06,NaN
1997,112007.500000,NaN,2.57986e+06,NaN
1998,115692.000000,NaN,2.70405e+06,NaN
1999,121003.500000,NaN,2.73575e+06,NaN
2000,125962.416667,NaN,2.81797e+06,NaN
2001,127369.583333,2.97396e+08,2.87753e+06,NaN
2002,130712.916667,2.94636e+08,2.90105e+06,NaN
2003,134868.083333,2.95376e+08,2.92154e+06,NaN
2004,142466.750000,3.20861e+08,2.9741e+06,NaN


In [38]:
# Fix houston population concatenation problem
houston_pop = [0,0,0,0,1974324,1979589,1990408,1992464,1991817,1995773,2038721,2044612,2060477,2087334,2108946,2126032,2161593,2199391,2241826,2286908,2309544,2316750,2318573,2320268, 0]
data_mod_h['Houston_population'] = houston_pop

# Fix Chicago population concatenation problem
chicago_pop = [0,0,0,0,2891582,2874477,2847266,2819117,2786291,2752523,2720181,2703991,2697359,2697006,2698838,2708114,2719141,2725731,2727066,2724344,2716723,2711069,2701423,2693976, 0]
data_mod_c['Chicago_population'] = chicago_pop

In [39]:
# Clean data, drop all NA rows to honor the recorded Houston data.
data_mod_h_final = data_mod_h.dropna()
data_mod_h_final = data_mod_h_final.rename(columns = {'Houston':'Houston_house_price'})
data_mod_h_final.head()

,Houston_house_price,Real GDP (thousands of chained 2012 dollars),Houston_Total_Employment,Houston_population
Date,,,,
2001,127369.583333,2.97396e+08,2.87753e+06,1979589
2002,130712.916667,2.94636e+08,2.90105e+06,1990408
2003,134868.083333,2.95376e+08,2.92154e+06,1992464
2004,142466.750000,3.20861e+08,2.9741e+06,1991817
2005,149148.750000,3.26131e+08,3.07839e+06,1995773


In [40]:
# Clean data, drop all NA rows to honor the recorded Chicago data.
data_mod_c_final = data_mod_c.dropna()
data_mod_c_final = data_mod_c_final.rename(columns = {'Chicago':'Chicago_house_price'})
data_mod_c_final.head()

,Chicago_house_price,Real GDP (thousands of chained 2012 dollars),Chicago_Total_Employment,Chicago_population
Date,,,,
2001,192719.000000,5.14982e+08,5.45027e+06,2874477
2002,207235.166667,5.17295e+08,5.38696e+06,2847266
2003,222490.500000,5.23312e+08,5.38703e+06,2819117
2004,239666.416667,5.36662e+08,5.44623e+06,2786291
2005,260888.166667,5.48632e+08,5.52826e+06,2752523
